In [ ]:
%%capture
import os
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
! pip install cvxpy
import cvxpy as cp
from tqdm import tqdm
import pandas as pd
import networkx as nx
from collections import Counter

In [ ]:
actual_path = os.getcwd()
os.chdir('/home/onyxia/work/advanced_ml_proj/data')
original_data = pd.read_csv('OnlineNewsPopularity.csv')
os.chdir(actual_path)

In [ ]:
full_names = []
for name in list(original_data.columns):
    temp_name = name.split()
    full_names.append(temp_name[0])

In [ ]:
original_data.columns = full_names

In [ ]:
original_data.columns

In [ ]:
original_data.drop(['url','timedelta'],axis=1,inplace=True)

In [ ]:
original_data['shares'].describe()

In [ ]:
n = original_data.shape[0]
original_data['pop_cat'] = original_data['shares'].copy()
data = original_data.copy()
for i in range(n):
    if data.loc[i,'shares'] < 946.:
        data.loc[i,'pop_cat'] = 0
    elif data.loc[i,'shares'] < 1400. :
        data.loc[i,'pop_cat'] = 1
    elif data.loc[i,'shares'] < 2800. :
        data.loc[i,'pop_cat'] = 2
    elif data.loc[i,'shares'] >= 2800. :
        data.loc[i,'pop_cat'] = 3

In [ ]:
def gaussian_ker(x, y, q):
    """a function to compute the gaussian kernel of two points
    -------------------------------
    inputs : 

    x : array-like, vector
    first vector for which we want to compute the kernel

    y : array-like, vector
    second vector for which we want to compute the kernel

    q : positive float,
    value of the bandwidth of the kernel

    returns:
    ker : float,
    the value of the kernel
    -------------------------------
    """

    ker = np.exp(-q*np.linalg.norm(x-y)**2)
    return ker

In [ ]:
def gram_mat(X, q):
    """
    a function to compute the gram matrix of a given dataset

    ----------------------------------------
    inputs : 
    X : array-like object, must be 2D
    the data for which we want to compute the gram matrix

    q : positive float, 
    the bandwidth of the gaussian kernel
    -----------------------------------------

    returns:
    K : the gram matrix
    """
    
    norms = np.linalg.norm(X, axis=1)**2
    dot = X@X.T
    squared_euclidian_distances = norms[:, None] - 2 * dot + norms[None, :]
    K = np.exp(-squared_euclidian_distances*q)
    return K

In [ ]:
def compute_seg(x, y, nb=20):
    """
    a function used to compute the segment between two points
    -----------------------------------
    Parameters : 

    x : array-like obj,
    an input, d>=2

    y : array-like obj, 
    the second input

    nb : int, 
    the number of points we want to have between the two points

    Returns : 

    segment : array-like
    an array of shape d (dimension of x), nb
    ---------------------------------------
    """
    d = x.shape[0]
    segment = np.zeros((nb, d))
    points = np.linspace(start=0., stop=1., num=nb, endpoint=True)
    
    for i in range(nb):
        t = points[i]
        segment[i, :] = (1-t) * x + t * y
        
    return segment

In [ ]:
def radius(x, sample, beta, bkb, q, ker_self=1.):
    """
    compute the radius for a given instance x
    -----------------------------------------------
    Parameters : 

    x : 1-D vector,
    the input vector

    sample : matrix, 
    the whole sample, 

    beta : array-like, 
    the calculated beta

    bkb : float,
    the result of beta.T@K@beta

    ker_self : float,
    the value of the kernel of the selected instance with itself, 
    set to 1 by default as we use mostly the gaussian kernel

    returns :

    radius : float,
    the distance between the test instance and the center of the 
    sphere enclosing all the points in the Hilbert space
    -----------------------------------------------
    """
    nb_samp = sample.shape[0]
    temp_k = np.zeros(nb_samp)

    for elem in range(nb_samp):
        temp_k[elem] = gaussian_ker(x, sample[elem,:], q=q)
        
    return np.sqrt(ker_self - 2*np.dot(temp_k, beta) + bkb)

In [ ]:
target = data['pop_cat'].copy()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#we center the data for the pca (as done in the article)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.drop('pop_cat', axis=1))


n_components = 15  
pca = PCA(n_components=n_components)

principal_components = pca.fit_transform(data_scaled)
columns = [f"PC{i+1}" for i in range(n_components)]
principal_df = pd.DataFrame(data=principal_components, columns=columns)


print("part of variance explained by every component :", pca.explained_variance_ratio_)


print("total variance explained :", np.sum(pca.explained_variance_ratio_))

d = principal_df.shape[1]
principal_df = np.hstack((principal_df,target.to_numpy().reshape(-1,1)))
principal_df_whole = principal_df.copy()
principal_df = pd.DataFrame(principal_df)

In [ ]:
#we rename the columns (more of a commidity than a necessity)
colnames = []
for i in range(15):
    colnames.append(f'pc_{i}')
colnames.append('target')
principal_df.columns = colnames
principal_df = principal_df.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.01,random_state=87876899))
target = principal_df['target'].copy()
print(principal_df.shape)

In [ ]:
principal_df.drop('target',axis=1,inplace=True)

In [ ]:
#a quick plot to see to what extent does the pca contributes to the possibility of 
#separation of the data (here on the first two components) 
plt.scatter(principal_df.iloc[:, 0], - principal_df.iloc[:, 1], marker='x', color='blue', s=20, linewidths=0.5)
plt.show()

In [ ]:
# Hyperparameters of the SVC procedure
N = n
q = 2
p = 0.1
C = 1

In [ ]:
gram_x = gram_mat(X=principal_df, q=q)

In [ ]:
n = len(principal_df)
beta = cp.Variable(n)
gram_x += gram_x.T
gram_x /= 2
gram_x = cp.psd_wrap(gram_x)


# Formulation de l'objectif
objective = cp.Maximize(cp.sum(np.ones(n) @ beta) - cp.quad_form(beta, gram_x))

# Contraintes
constraints = [
    beta >= 0,  # 0 <= beta_j
    beta <= C,  # beta_j <= C
    cp.sum(beta) == 1  # La somme des éléments de beta doit être égale à 1
]

# Définir le problème d'optimisation
problem = cp.Problem(objective, constraints)

# Résoudre le problème
problem.solve(solver='CLARABEL')

In [ ]:
true_beta = beta.value
beta_k_beta = true_beta.T @ gram_x @ true_beta

In [ ]:
index_of_sv = []
index_of_bsv = []
N = principal_df.shape[0]
for i in range(N):
    if 1e-10 < true_beta[i] and true_beta[i] < C:
        index_of_sv.append(i)
    elif true_beta[i] >= C - 1e-3:
        index_of_bsv.append(i)

print('index of sv', index_of_sv)
print('number of sv', len(index_of_sv))
print('index of bsv', index_of_bsv)
print('number of bsv', len(index_of_bsv))

In [ ]:
potential_sv = principal_df.iloc[index_of_sv, :].to_numpy()

In [ ]:
gram_x = gram_mat(X=principal_df, q=q)
gram_x += gram_x.T
gram_x /= 2
r = []
beta_k_beta = true_beta.T @ gram_x @ true_beta
for point_sv in potential_sv:
    temp_K = np.zeros(N)
    for elem in range(N):
        temp_K[elem] = gaussian_ker(point_sv, principal_df.iloc[elem, :], q=q)
    r_xi = np.sqrt(1 - 2 * np.dot(temp_K, true_beta) + beta_k_beta)
    r.append(r_xi)
rad = np.mean(r)

In [ ]:
n = principal_df.shape[0]
adjacency_mat = np.zeros((n, n))

for i in tqdm(range(n)):
    for j in range(i+1, n):
        decision = True
        segment = compute_seg(x=principal_df.to_numpy()[i,:], y=principal_df.to_numpy()[j,:])
        list_of_val = []
        for point in segment:
            dist = radius(x=point, beta=true_beta, bkb=beta_k_beta, sample=principal_df.to_numpy(), q=q)
            list_of_val.append(dist)
        for value in list_of_val:
            if value > rad:
                decision = False
        
        if decision == True:
            adjacency_mat[i, j] = 1

adjacency_mat = adjacency_mat + adjacency_mat.T
adjacency_mat /= 2
for i in range(n):
    adjacency_mat[i,i] = 0

In [ ]:
G = nx.from_numpy_array(adjacency_mat)

clusters = list(nx.connected_components(G))
print('number of clusters detected', len(clusters))

In [ ]:

sorted_clusters = sorted(clusters, key=len, reverse=True)
print(f"largest cluster: {len(sorted_clusters[0])}")

#a quick preview of how we are doing
top_5_clusters = sorted_clusters[:5]
for i, cluster in enumerate(top_5_clusters, start=1):
    print(f"size of cluster {i}: {len(cluster)}")


In [ ]:
#cluster labelling
misclassified_count = 0
for cluster in clusters:

    if len(cluster) > 1:

        cluster_labels = [target[i] for i in cluster]
        majority_label = Counter(cluster_labels).most_common(1)[0][0]
        for i in cluster:
            if target[i] != majority_label:
                misclassified_count += 1

print(f"total number of missclassification: {misclassified_count}")
